In [ ]:
import torch
import numpy as np
from torch import nn
from scipy.sparse import csr_matrix

# https://stackoverflow.com/questions/52299420/scipy-csr-matrix-understand-indptr

In [ ]:
class myGraphSAINT(nn.Module):
    def __init__(self):
        super(myGraphSAINT, self).__init__()
    
    def forward(self, x):
        return x
    
    def sampleGraph(self, graph):
        pass

In [ ]:
x = torch.ones(3)
model = myGraphSAINT()
print(model(x))

## Data loading and processing

In [ ]:
adj_train = np.load("./ppi/adj_train.npz")

# The data for the (symmetric) adjacency matrix
data = adj_train["data"]
indices = adj_train["indices"]
indptr = adj_train["indptr"]
shape = adj_train["shape"]

# # This is a less memory-efficient method to get the sparse torch adjacency matrix, that also requires SciPy
# adj_matrix = csr_matrix((data, indices, indptr), shape=shape).toarray().astype(int)
# np.set_printoptions(threshold=sys.maxsize)
# adj_matrix = torch.from_numpy(adj_matrix).to_sparse()

# Change the SciPy csr format to torch format
torch_first_indices = []
for i in range(len(indptr)-1):
    torch_first_indices += [i for ind in indices[indptr[i]:indptr[i+1]]]
torch_first_indices = np.asarray(torch_first_indices)
torch_indices = np.stack((torch_first_indices, indices))

# The given shape for the ppi train set is much larger than the number of actual nodes in the set
num_nodes = len(np.unique(torch_indices))
shape_small = [num_nodes, num_nodes]

# Create the adjacency matrix
adj_matrix = torch.sparse_coo_tensor(indices=torch_indices, values=data.astype(int), size=shape_small, dtype=torch.float64)

# Calculate the node degrees
degree = [0.0 for i in range(shape_small[0])]
for i in torch_indices.T:
    degree[i[0]] += 1
    if i[1] == i[0]:
        degree[i[0]] += 1
inverse_degree = np.reciprocal(np.asarray(degree))

# Calculate the normalized adjacency matrix
norm_adj_data = inverse_degree[torch_indices[0]]*data
norm_adj_matrix = torch.sparse_coo_tensor(indices=torch_indices, values=norm_adj_data.astype(int), size=shape_small,
                                          dtype=torch.float64)

## Sampling

In [ ]:
# IS THIS THE NORM???
torch.norm(torch.ones(3))**2